In [ ]:
import torch
import cv2
import numpy as np
from RT.models import EngineBuilder
from RT.models.cudart_api import TRTEngine
from RT.models.utils import blob
from RT.models.torch_utils import det_postprocess

In [ ]:
model_name = "models/face_detection_model.onnx"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
input_shape = (1, 3, 640, 640)

In [ ]:
builder = EngineBuilder(model_name, device)
builder.seg = True
builder.build(fp16=True,
              input_shape=list(input_shape),
              iou_thres=0.65,
              conf_thres=0.25,
              topk=100)

In [ ]:
def ImageBox(image, new_shape=(640, 640), color=(0, 0, 0)):
    
    width, height, channel = image.shape
    
    ratio = min(new_shape[0] / width, new_shape[1] / height)
    
    new_unpad = int(round(height * ratio)), int(round(width * ratio))
    
    dw, dh = (new_shape[0] - new_unpad[0])/2, (new_shape[1] - new_unpad[1])/2

    if (height, width) != new_unpad:
        image = cv2.resize(image, new_unpad, interpolation=cv2.INTER_LINEAR)
    
    top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
    
    image = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
    
    return image, ratio, (dw, dh)

In [ ]:
enggine = TRTEngine('models/face_detection_model.engine')
H, W = enggine.inp_info[0].shape[-2:]

In [ ]:
image = cv2.imread("image.jpg")
image, ratio, dwdh = ImageBox(image)

In [ ]:
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
tensor = blob(image, return_seg=False)
tensor = torch.asarray(tensor)

In [ ]:
dwdh = np.array(dwdh * 2, dtype=np.float32)

In [ ]:
results = enggine(tensor)

In [ ]:
results